# Testing Recommender Functions

This notebook installs the required dependencies and tests the recommender functions defined in your modules (e.g. `utils.py` and `recommender.py`). It loads the models via the `get_models()` function and then tests various ensemble methods.

In [ ]:
# Install required dependencies
!pip install --upgrade pip
!pip install numpy scikit-learn tensorflow keras fastapi torch transformers

# If your project has a requirements.txt file, you can also use:
# !pip install --no-cache-dir -r requirements.txt

In [ ]:
# Import required modules
import numpy as np
from utils import get_models, tokenize_input  # Ensure these are in your PYTHONPATH
from recommender import ensemble_bagging, ensemble_boosting, train_stacking_meta_model, ensemble_stacking, hybrid_ensemble

# For demonstration, we assume get_models() returns a dictionary of models for clusters 0, 1, 2, etc.
print("Modules imported successfully.")

In [ ]:
# Load ensemble models using the get_models function
print("Loading models...")
models_dict = get_models()
print("Models loaded:", models_dict.keys())

# Define a test input text for recommendation
test_input = "This is a dummy test input for the recommender functions."

# Test ensemble bagging
bagging_pred = ensemble_bagging(test_input, models_dict)
print("Ensemble Bagging Prediction:", bagging_pred)

# Test ensemble boosting with dummy error values
dummy_errors = np.array([0.2, 0.15, 0.25])
boosting_pred = ensemble_boosting(test_input, models_dict, dummy_errors)
print("Ensemble Boosting Prediction:", boosting_pred)

# Test ensemble stacking with dummy training data
X_train_dummy = np.array([
    [0.80, 0.75, 0.85],
    [0.55, 0.60, 0.50],
    [0.30, 0.35, 0.25],
    [0.20, 0.25, 0.15]
])
y_train_dummy = np.array([1, 0, 1, 0])
meta_model = train_stacking_meta_model(X_train_dummy, y_train_dummy)
stacking_pred = ensemble_stacking(test_input, models_dict, meta_model)
print("Ensemble Stacking Prediction:", stacking_pred)

# Test hybrid ensemble
hybrid_pred = hybrid_ensemble(test_input, models_dict, dummy_errors, meta_model)
print("Hybrid Ensemble Prediction:", hybrid_pred)

### Next Steps

You can now develop and test your recommendation functions independently of the FastAPI backend. 

For further debugging, you might want to add additional print statements or assertions within your recommender functions.